In [1]:
import os


os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM


MODEL_ID = "/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000"

model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


/root/work/axolotl/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.quantization import QuantizationModifier


# Configure the simple PTQ quantization
recipe = QuantizationModifier(targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# Apply the quantization algorithm.
oneshot(model=model, recipe=recipe)

# Save the model.
SAVE_DIR = '/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8'
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)

/root/work/axolotl/env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:186: UserWarning: Field name "registry_requires_subclass" shadows an attribute in parent "RegistryMixin"; 
  warnings.warn(
/root/work/axolotl/env/lib/python3.10/site-packages/pydantic/_internal/_fields.py:186: UserWarning: Field name "registry_requires_subclass" shadows an attribute in parent "SparsityCompressionConfig"; 
  warnings.warn(


2024-12-30T13:50:47.443708+0800 | main | WARNING - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
2024-12-30T13:50:47.446363+0800 | main | INFO - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
clear_sparse_session=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspee

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: k-l-lambda (k-l-lambda-org). Use `wandb login --relogin` to force relogin


2024-12-30T13:50:50.534610+0800 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers


2024-12-30T13:50:57.500141+0800 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 0 modifiers
2024-12-30T13:50:57.501784+0800 | populate_datasets | INFO - Running oneshot without calibration data. This is expected for weight-only and dynamic quantization


...

2024-12-30T13:51:08.650772+0800 | initialize | INFO - Compression lifecycle initialized for 1 modifiers


manager stage: Modifiers finalized


2024-12-30T13:51:08.652503+0800 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2024-12-30T13:51:08.662381+0800 | get_model_compressor | INFO - Inferring a sparsity configuration requires a global sparsity calculation. This can be costly for large models. To skip the calculation of compression statistics set skip_compression_stats=True


Calculating model sparsity: 100%|██████████| 371/371 [00:04<00:00, 79.07it/s] 
Calculating quantization compression ratio: 180it [00:00, 968.50it/s]
Quantized Compression: 100%|██████████| 371/371 [00:01<00:00, 254.77it/s]


('/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8/tokenizer_config.json',
 '/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8/special_tokens_map.json',
 '/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8/tokenizer.json')

In [4]:
from safetensors import safe_open


state = dict()

with safe_open("/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8/model.safetensors", 'pt', device='cpu') as f:
	for key in f.keys():
		state[key] = f.get_tensor(key)

for k, t in state.items():
	print(k, t.shape, t.dtype)

lm_head.weight torch.Size([128256, 2048]) torch.bfloat16
model.embed_tokens.weight torch.Size([128256, 2048]) torch.bfloat16
model.layers.0.input_layernorm.weight torch.Size([2048]) torch.bfloat16
model.layers.0.mlp.down_proj.weight torch.Size([2048, 8192]) torch.float8_e4m3fn
model.layers.0.mlp.down_proj.weight_scale torch.Size([2048, 1]) torch.bfloat16
model.layers.0.mlp.gate_proj.weight torch.Size([8192, 2048]) torch.float8_e4m3fn
model.layers.0.mlp.gate_proj.weight_scale torch.Size([8192, 1]) torch.bfloat16
model.layers.0.mlp.up_proj.weight torch.Size([8192, 2048]) torch.float8_e4m3fn
model.layers.0.mlp.up_proj.weight_scale torch.Size([8192, 1]) torch.bfloat16
model.layers.0.post_attention_layernorm.weight torch.Size([2048]) torch.bfloat16
model.layers.0.self_attn.k_proj.weight torch.Size([512, 2048]) torch.float8_e4m3fn
model.layers.0.self_attn.k_proj.weight_scale torch.Size([512, 1]) torch.bfloat16
model.layers.0.self_attn.o_proj.weight torch.Size([2048, 2048]) torch.float8_e4m3f

In [5]:
del state['lm_head.weight']

In [6]:
from safetensors.torch import save_file


save_file(state, "/models/train/20241224-llama3.2-1B-finetune-l1-1d9e/checkpoint-5000-FP8/model.safetensors", metadata={'format': 'pt'})

In [9]:
import torch


size = 0

for k, t in state.items():
	ts = t.numel() * (2 if t.dtype == torch.bfloat16 else 1)
	print(k, ts)
	size += ts

size

lm_head.weight 65667072
model.embed_tokens.weight 65667072
model.layers.0.input_layernorm.weight 512
model.layers.0.mlp.down_proj.weight 262144
model.layers.0.mlp.down_proj.weight_scale 512
model.layers.0.mlp.gate_proj.weight 262144
model.layers.0.mlp.gate_proj.weight_scale 2048
model.layers.0.mlp.up_proj.weight 262144
model.layers.0.mlp.up_proj.weight_scale 2048
model.layers.0.post_attention_layernorm.weight 512
model.layers.0.self_attn.k_proj.weight 16384
model.layers.0.self_attn.k_proj.weight_scale 128
model.layers.0.self_attn.o_proj.weight 524288
model.layers.0.self_attn.o_proj.weight_scale 512
model.layers.0.self_attn.q_proj.weight 524288
model.layers.0.self_attn.q_proj.weight_scale 4096
model.layers.0.self_attn.v_proj.weight 16384
model.layers.0.self_attn.v_proj.weight_scale 128
model.layers.1.input_layernorm.weight 512
model.layers.1.mlp.down_proj.weight 262144
model.layers.1.mlp.down_proj.weight_scale 512
model.layers.1.mlp.gate_proj.weight 262144
model.layers.1.mlp.gate_proj.w

138847744